In [21]:
from pony.orm import *

db = Database()

Import pony and set up the database

In [22]:
class Person(db.Entity):
    name = Required(str)
    age = Required(int)
    cars = Set('Car')
    
    
class Car(db.Entity):
    make = Required(str)
    model = Required(str)
    owner = Required(Person)
    
# Simple check to show the table format
show(Person)

class Person(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    age = Required(int)
    cars = Set(Car)


The next step is to create the actual database and its tables. Below it shows how this could be done in both memory or creating a file. If you are creating the database from a shell you need to give the fully path to the database. I am not sure if this is the same from a script

In [23]:
# db.bind('sqlite', ':memory:')
# or use
db.bind('sqlite', '/home/boomatang/Projects/Python/Cullen-DMU/db-test.sqlite', create_db=True)

The next required step is to make a mpping of the database objects. This most be done after the tables are created.

In [24]:
db.generate_mapping(create_tables=True)

For the use case here we will turn on the debug mode so as to see the sql that is be transacted


In [25]:
sql_debug(True)


Next we create some entity instances and commit them to the database

In [26]:
p1 = Person(name='John', age=20)
p2 = Person(name='Mary', age=22)
p3 = Person(name='Bob', age=30)
c1 = Car(make='Toyota', model='Prius', owner=p2)
c2 = Car(make='Ford', model='Explorer', owner=p3)
commit()


Whenworking outside of the interactive shell you will be required to have your code in side the db_session. This can be done by using **@db_session()** decorator

In [27]:
@db_session
def print_person_name(person_id):
    p = Person[person_id]
    print(p.name)
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

@db_session
def add_car(person_id, make, model):
    Car(make=make, model=model, owner=Person[person_id])
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

The db_session can also be done with using the **with** function as shown below


In [28]:
with db_session:
    p = Person(name='Kate', age=33)
    Car(make='Audi', model='R8', owner=p)
    # commit() will be done automatically
    # dataabase session cache will be cleared automatically
    # database connection will be returned to the pool

The select function uses a generator to pull data from the database. Below are a few examples of how to select data from the database


In [29]:
select(p for p in Person if p.age > 20)

In [30]:
select(p for p in Person if p.age>20)[:]


[Person[2], Person[3], Person[4]]

In [32]:
select(p for p in Person).order_by(Person.name)[:2]


[Person[3], Person[1]]

In [33]:
select(p for p in Person).order_by(Person.name)[:2].show()

id|name|age
--+----+---
3 |Bob |30 
1 |John|20 
